## Recomendação utilizando KNN

Este notebook foi destinado para o treinmento de um modelo KNN para identificar similaridades lógicas entre os municípios. Por fim, os dados utilizados foram os dados meteorológicos, solo e recusos hídricos.

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.impute import KNNImputer
from geopy.distance import geodesic
import joblib as jb

# Classificadores:
from sklearn.neighbors import NearestNeighbors

In [25]:
def dataset_recomendacao():
    df_municipios_sertao = pd.read_csv(r"..\Dados\Views\municipios_sertao.csv", index_col=0)
    df_meteorologia = pd.read_csv("..\Dados\Views\dados_meteorologicos.csv", index_col=0)
    df_solo = pd.read_csv(r"..\Dados\Views\solos_municipios.csv", index_col=0)
    df_recursos_hidricos = pd.read_csv(r"..\Dados\Views\recursos_hidricos.csv", index_col=0)
    df_ql = pd.read_csv(r"..\Dados\Views\qualidade_da_agua.csv", index_col=0)
    df_classificador = df_municipios_sertao[["IBGE7", "NOME"]].copy()
    
    # Merges com o dataframe do classificador:
    df_classificador = df_classificador.merge(df_meteorologia, on="IBGE7", how="left")
    df_classificador = df_classificador.merge(df_recursos_hidricos[["IBGE7", "AREA_IRRIGADA_TOT_POT_E", "AGUA_SOLO"]], on="IBGE7", how="left")
    df_classificador = df_classificador.merge(df_ql, on="IBGE7", how="left")

    # Obtendo solo de maior área dentro do município
    df_solo_esparsa = df_solo.pivot_table(index="IBGE7", columns="SOLO", values="AREA_TOTAL").fillna(value=0).reset_index()
    df_classificador = df_classificador.merge(df_solo_esparsa, how="inner", on="IBGE7")
    return df_classificador

  
def recomendacao_preprocessamento(dataset):
    X = dataset.drop(columns=["IBGE7", "NOME"])
    cat_columns = [i for i in X.columns if i in X.loc[:, "AGUA":].columns]
    colunas_log = [i for i in X.columns if (i not in cat_columns)&(i not in ["LATITUDE", "LONGITUDE"])]
    
    # Aplicando log transformation para conter a variabilidade dos dados:
    log_transformation = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)
    X[colunas_log] = log_transformation.transform(X[colunas_log])

    # Aplicando MinMax scaler para conter o range dos dados:
    min_max = MinMaxScaler()
    colunas_totais = X.columns
    X[colunas_totais] = min_max.fit_transform(X[colunas_totais])

    # Aplicando imputação dos dados:
    imputer = KNNImputer(n_neighbors=5)
    X_imputado = pd.DataFrame(imputer.fit_transform(X))
    X_imputado.columns = X.columns
    return X_imputado



def filtro_recomendacao(vizinhos, nome_potencial_mun):
    df_produto_agricola = pd.read_csv(r"..\Dados\Tabela_final\dados_producao_agricola.csv", index_col=0)
    
    # Calculo do Valor da Produção por área plantada para avaliar se há subexploração:
    df_produto_agricola["VALOR_POR_AREA_PLANTADA"] = df_produto_agricola["VALOR_PROD"] / df_produto_agricola["AREA_PLANTADA"]

    # Produtos produzidos:
    df_cidade_escolhida = df_produto_agricola[df_produto_agricola["NOME"] == nome_potencial_mun].sort_values("VALOR_PROD", ascending=False)
    
    # Rendimento por área dos vizinhos lógicos de ITAPICURU:
    df_agrupado = df_produto_agricola[df_produto_agricola["NOME"].isin(vizinhos["NOME"].values)]

    # Média do Valor produzido dos produtos cultivados pelos vizinhos:
    df_agrupado = df_agrupado.groupby("PRODUTO").agg({"VALOR_PROD":"mean",
                                                  "AREA_PLANTADA":"mean",
                                                   "VALOR_POR_AREA_PLANTADA":"mean"}).sort_values("VALOR_PROD", ascending=False)
    

    df_agrupado.rename(columns={"VALOR_PROD": "VALOR_PROD_MEDIA_VIZ",
                                "AREA_PLANTADA":"AREA_PLANTADA_MED_VIZ", 
                                "VALOR_POR_AREA_PLANTADA":"VALOR_POR_AREA_PLANTADA_MEDIA_VIZ"},inplace=True)
    
    df_agrupado = df_agrupado.reset_index()

    # Seleção dos produtos mais interessantes:
    df_comparativo = df_agrupado.merge(df_cidade_escolhida[["PRODUTO","VALOR_PROD", 
                                                                    "VALOR_POR_AREA_PLANTADA"]], on="PRODUTO", how="left")
    df_comparativo.rename(columns={"VALOR_PROD":f"VALOR_PROD_{nome_potencial_mun}", 
                                "VALOR_POR_AREA_PLANTADA":f"VALOR_POR_AREA_PLANT_{nome_potencial_mun}"}, inplace=True)

    df_comparativo = df_comparativo.fillna(0)
    nome_coluna_selecao1 = f"VALOR_PROD_{nome_potencial_mun}"
    nome_coluna_selecao2 = f"VALOR_POR_AREA_PLANT_{nome_potencial_mun}"
    condicao_selecao1 = (df_comparativo[nome_coluna_selecao1] < df_comparativo["VALOR_PROD_MEDIA_VIZ"])
    condicao_selecao2 = (df_comparativo[nome_coluna_selecao2] < df_comparativo["VALOR_POR_AREA_PLANTADA_MEDIA_VIZ"])    
    return df_comparativo[condicao_selecao1&condicao_selecao2] 

## 1) Carregando dataset de recomendação

In [57]:
df_classificador = dataset_recomendacao()
df_solo = pd.read_csv(r"..\Dados\Views\solos_municipios.csv", index_col=0)
df_municipios_sertao = pd.read_csv(r"..\Dados\Views\municipios_sertao.csv", index_col=0)

## 2) Identificar vizinhos Lógicos utilizando Nearest Neighbours

### 2.1) Pré-processamento

Iremos primeiro retirar colunas que não serão úteis para o treinamento e fazere o pré-processamento com uma transformação logarítma para diminuir a variabilidade e o efeito de outliers. Além disso, um min max scale foi aplicado após, para delimitar o range das variáveis entre 0 e 1.

In [58]:
X_imputado = recomendacao_preprocessamento(df_classificador)

### 2.2) Nearest Neighbours

Iremos treinar um KNearest Neighbour com número de vizinhos igual a 5 e com a métrica de similaridade "cosino".

In [59]:
k_neighburs = 5 
knn = NearestNeighbors(n_neighbors=k_neighburs, n_jobs=-1, metric="cosine")
knn.fit(X_imputado)

NearestNeighbors(metric='cosine', n_jobs=-1)

In [60]:
# Obtendo os vizinhos para todos os dados de treino:
distance, neighbours_indices = knn.kneighbors(X_imputado, n_neighbors=k_neighburs+1)

In [61]:
# Escolhendo o município de interesse para identificar seus vizinhos:
index = df_classificador[df_classificador["NOME"] == "ITAPICURU"].index[0]
nomes = df_classificador.loc[neighbours_indices[index]]["NOME"].values
dataframe = pd.DataFrame({"Municípios":nomes[1:], "Distancias":distance[0][1:]})
dataframe

,Municípios,Distancias
0,ITAOBIM,0.002534
1,AGUA FRIA,0.003839
2,ACARAU,0.003918
3,RIBEIRA DO POMBAL,0.004334
4,RIBEIRA DO AMPARO,0.004397


Podemos perceber que seus vizinhos possuem muita similaridade em questões meteorológicas e possuem distribuições de solos similares.

### 2.3) Recomendação Lógica

A função abaixo, utiliza dos dados recomendados dos vizihos lógicos para retornar os produtos as quais o município em questão está subexplorado ou que não produz em comparação com os vizinhos lógicos.

In [79]:
vizinhos = df_classificador.loc[neighbours_indices[index]]
recomendacao = filtro_recomendacao(vizinhos, nome_potencial_mun="ITAPICURU")
recomendacao

,PRODUTO,VALOR_PROD_MEDIA_VIZ,AREA_PLANTADA_MED_VIZ,VALOR_POR_AREA_PLANTADA_MEDIA_VIZ,VALOR_PROD_ITAPICURU,VALOR_POR_AREA_PLANT_ITAPICURU
0,MELAO,22705.166667,755.333333,26.275897,4674.666667,20.900149
3,MELANCIA,3078.466667,138.933333,17.332770,880.333333,6.586035
7,BATATA-DOCE,757.666667,34.666667,21.855769,0.000000,0.000000
8,MANGA,460.166667,28.000000,12.310462,0.000000,0.000000
10,AMENDOIM (EM CASCA),113.000000,50.000000,2.260000,0.000000,0.000000
11,CAFE (EM GRAO) CANEPHORA,4.333333,0.666667,6.500000,0.000000,0.000000
12,CAFE (EM GRAO) TOTAL,4.333333,0.666667,6.500000,0.000000,0.000000


Conclusões:
- ITAPICURU está abaixo da média comparada com seus semelhantes em questão de Valor de Produção: 
     1. Melão - 4674.666667
     2. Melancia - 880.333333
     
- Dado os produtos em que ITAPICURU produz abaixo da média de seus semelhantes, percebemos que o Melão, Melancia têm um valor por área médio de seus semelhantes bem superior ao de ITAPICURU (Melão=20.900149, Melancia=6.586035). Sendo o Melão o produto mais interessante para ser cultivado.

- Foi indicado pelo algoritmo, e que não é produzido em ITAPICURU, a produção de MANGA, AMENDOIM (EM CASCA), BATATA-DOCE, CAFE (EM GRAO) CANEPHORA e CAFE (EM GRAO) TOTAL.

## 3) Similaridade Geográfica a partir da distância

In [48]:
# Função para calcular a distância em KM, entre duas coordenadas geográficas:
def distancia_circular(df_municipios_sertao, threhold, municipio_start):
    dicionario_distancias = {}
    lista_distancias = []
    municipio_escolhido = df_municipios_sertao[df_municipios_sertao["NOME"] == municipio_start][["LATITUDE", "LONGITUDE"]]
    for municipio_qualquer, row2 in df_municipios_sertao[["LATITUDE", "LONGITUDE"]].iterrows():
        nome = df_municipios_sertao.loc[municipio_qualquer, "NOME"]
        uf = df_municipios_sertao.loc[municipio_qualquer, "UF"]
        ibge = df_municipios_sertao.loc[municipio_qualquer, "IBGE7"]
        if municipio_start != nome:
            distancia = geodesic(municipio_escolhido.values, row2.values).kilometers
            if distancia <= threhold:
                lista_distancias.append((ibge, nome, uf,distancia))
    
    return pd.DataFrame(lista_distancias, columns=["IBGE7", "NOME", "UF", "DISTANCIAS"])

### 3.1) ITAPICURU vizinhos físicos

In [49]:
# Escolha do município a ser avaliado
nome_potencial_mun = "ITAPICURU"
codigo_mun_potencial = df_municipios_sertao[df_municipios_sertao["NOME"] == nome_potencial_mun]["IBGE7"].values[0]
criterio_distancia = 50

In [52]:
# Calculo das distâncias deste município até otros:
municipios_vizinhos = distancia_circular(df_municipios_sertao, threhold=criterio_distancia, municipio_start=nome_potencial_mun)
municipios_vizinhos

,IBGE7,NOME,UF,DISTANCIAS
0,2907905,CIPO,BA,39.141885
1,2909604,CRISOPOLIS,BA,23.277050
2,2922904,NOVA SOURE,BA,29.698246
3,2923100,OLINDINA,BA,13.005937
4,2926509,RIBEIRA DO AMPARO,BA,36.578826
5,2807402,TOBIAS BARRETO,SE,28.573107


In [56]:
recomendacao = filtro_recomendacao(municipios_vizinhos, nome_potencial_mun)
recomendacao

,PRODUTO,VALOR_PROD_MEDIA_VIZ,AREA_PLANTADA_MED_VIZ,VALOR_POR_AREA_PLANTADA_MEDIA_VIZ,VALOR_PROD_ITAPICURU,VALOR_POR_AREA_PLANT_ITAPICURU
0,MELAO,40735.666667,1287.000000,31.651645,4674.666667,20.900149
4,MELANCIA,1254.533333,64.600000,14.239388,880.333333,6.586035
5,TOMATE,371.333333,3.666667,101.272727,0.000000,0.000000
8,AMENDOIM (EM CASCA),96.277778,33.055556,3.083810,0.000000,0.000000
9,MANGA,10.444444,3.222222,4.011111,0.000000,0.000000


Conclusões:

- ITAPICURU possui um Valor de Produção e área plantada do Melão, Melancia abaixo da média ao seu entorno. Sendo o Melão o produto mais interessante para ser explorado.
- Segundo a proximidade física, há municípios que plantam produtos que não são produzidos por ITAPICURU como o TOMATE, AMENDOIM (EM CASCA) e a MANGA.

### 3.2) Análise de solos dos vizinhos físicos

In [68]:
# Solos que podemos encontrar em ITAPICURU:
df_solo[df_solo["IBGE7"].isin([codigo_mun_potencial])].merge(df_municipios_sertao[["NOME", "UF", "IBGE7"]], on="IBGE7", how="inner")

,IBGE7,SOLO,AREA_TOTAL,NOME,UF
0,2916500,LAd - Latossolos Amarelos Distroficos,18032.365183,ITAPICURU,BA
1,2916500,PVAd - Argissolos Vermelho-Amarelos Distroficos,46579.916148,ITAPICURU,BA
2,2916500,RQo - Neossolos Quartzarenicos Orticos,86920.869936,ITAPICURU,BA
3,2916500,SNo - Planossolos Natricos Orticos,15514.030117,ITAPICURU,BA


In [69]:
# Solos dos vizinhos lógicos identificados:
cod_mun_viz_fis = municipios_vizinhos["IBGE7"].values
df_municipios_sertao[["NOME", "UF", "IBGE7"]].merge(df_solo[df_solo["IBGE7"].isin(cod_mun_viz_fis)], on="IBGE7", how="inner")

,NOME,UF,IBGE7,SOLO,AREA_TOTAL
0,CIPO,BA,2907905,LAd - Latossolos Amarelos Distroficos,11816.751307
1,CIPO,BA,2907905,RQo - Neossolos Quartzarenicos Orticos,2501.373861
2,CRISOPOLIS,BA,2909604,LAd - Latossolos Amarelos Distroficos,10644.589008
3,CRISOPOLIS,BA,2909604,PVAd - Argissolos Vermelho-Amarelos Distroficos,21211.041084
4,CRISOPOLIS,BA,2909604,PVAe - Argissolos Vermelho-Amarelos Eutroficos,32234.030849
5,NOVA SOURE,BA,2922904,LAd - Latossolos Amarelos Distroficos,73498.034469
6,NOVA SOURE,BA,2922904,PVAe - Argissolos Vermelho-Amarelos Eutroficos,25392.652704
7,OLINDINA,BA,2923100,LAd - Latossolos Amarelos Distroficos,42378.369006
8,OLINDINA,BA,2923100,PVAd - Argissolos Vermelho-Amarelos Distroficos,1322.188363
9,OLINDINA,BA,2923100,PVAe - Argissolos Vermelho-Amarelos Eutroficos,11534.945980


Conclusões:
- ITAPICURU, possui maior área e as mesmas tipologias de solo, que o maior produtor de Melão vizinho, entretanto, sua produção é bem menor em comparação a ele, o que corrobora para a subexploração dado que temos as mesmas condições mas produzimos menos.